<a href="https://colab.research.google.com/github/existinmyhead/Winter22/blob/main/CIS3200/Project2/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyspark

!wget -q 'https://drive.google.com/uc?export=download&id=1wAdLRbT_YgWJ1XRl39JM-lKwJpewdwyM' -O 'Sample.txt'

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 53.6 MB/s 


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('SparkWordCount')
sc = SparkContext.getOrCreate(conf = conf)

sqlContext = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
input_file = sc.textFile('Sample.txt')

input_file.collect()[:10]

['The Project Gutenberg EBook of The Complete Works of William Shakespeare, by',
 'William Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **',
 '**     Please follow the copyright guidelines in this file.     **']

In [ ]:
# import re
# flatMapVar = input_file.flatMap(lambda line: re.split(r"[ .\*\,\'\?\;\!\$\&\:]+", line))

# flatMapVar.collect()[:10]

import re
def countFivewords(ls, n = 5):
  lines = re.split(r"[ ,\.\*)\(\)\<\>\#\:\-\!\[\]]+", ls)
  holder = zip(*[lines[i:] for i in range(n)])
  return [" ".join(h) for h in holder]

flatMapVar = input_file.flatMap(lambda lines: countFivewords(lines)).map(lambda lines: (lines, 1)) 
                #  .reduceByKey(lambda a, b: a + b)

flatMapVar.collect()[:20]

[('The Project Gutenberg EBook of', 1),
 ('Project Gutenberg EBook of The', 1),
 ('Gutenberg EBook of The Complete', 1),
 ('EBook of The Complete Works', 1),
 ('of The Complete Works of', 1),
 ('The Complete Works of William', 1),
 ('Complete Works of William Shakespeare', 1),
 ('Works of William Shakespeare by', 1),
 ('This eBook is for the', 1),
 ('eBook is for the use', 1),
 ('is for the use of', 1),
 ('for the use of anyone', 1),
 ('the use of anyone anywhere', 1),
 ('use of anyone anywhere at', 1),
 ('of anyone anywhere at no', 1),
 ('anyone anywhere at no cost', 1),
 ('anywhere at no cost and', 1),
 ('at no cost and with', 1),
 ('almost no restrictions whatsoever You', 1),
 ('no restrictions whatsoever You may', 1)]

In [ ]:
# keyPairs = flatMapVar.map(lambda word: (word, 1))

# keyPairs.collect()[:10]

In [ ]:
countKeys = flatMapVar.reduceByKey(lambda a, b: a + b)

countKeys.collect()[:20]

[('The Project Gutenberg EBook of', 1),
 ('EBook of The Complete Works', 2),
 ('Complete Works of William Shakespeare', 3),
 ('This eBook is for the', 2),
 ('eBook is for the use', 2),
 ('for the use of anyone', 2),
 ('use of anyone anywhere at', 2),
 ('of anyone anywhere at no', 2),
 ('anywhere at no cost and', 2),
 ('at no cost and with', 2),
 ('restrictions whatsoever You may copy', 2),
 ('whatsoever You may copy it', 2),
 ('copy it give it away', 2),
 ('re use it under the', 2),
 ('use it under the terms', 2),
 ('under the terms of the', 2),
 ('the terms of the Project', 2),
 ('of the Project Gutenberg License', 2),
 ('or online at www gutenberg', 2),
 ('online at www gutenberg org', 2)]

In [ ]:
countsDF = sqlContext.createDataFrame(countKeys) \
            .withColumnRenamed('_1','Word').withColumnRenamed('_2','Count')

countsDF.toPandas().to_csv('counts.csv', index=False)

countsDF.show()

+--------------------+-----+
|                Word|Count|
+--------------------+-----+
|The Project Guten...|    1|
|EBook of The Comp...|    2|
|Complete Works of...|    3|
|This eBook is for...|    2|
|eBook is for the use|    2|
|for the use of an...|    2|
|use of anyone any...|    2|
|of anyone anywher...|    2|
|anywhere at no co...|    2|
| at no cost and with|    2|
|restrictions what...|    2|
|whatsoever You ma...|    2|
|copy it give it away|    2|
| re use it under the|    2|
|use it under the ...|    2|
|under the terms o...|    2|
|the terms of the ...|    2|
|of the Project Gu...|    2|
|or online at www ...|    2|
|online at www gut...|    2|
+--------------------+-----+
only showing top 20 rows



Here's all the above steps merged into one block of code:

In [ ]:
input_file = sc.textFile('Sample.txt')

import re
def countFivewords(ls, n = 5):
  lines = re.split(r"[ ,\.\*)\(\)\<\>\#\:\-\!\[\]]+", ls)
  holder = zip(*[lines[i:] for i in range(n)])
  return [" ".join(h) for h in holder]

flatMapVar = input_file.flatMap(lambda lines: countFivewords(lines)).map(lambda lines: (lines, 1)) 

countKeys = flatMapVar.reduceByKey(lambda a, b: a + b)

countsDF = sqlContext.createDataFrame(countKeys) \
            .withColumnRenamed('_1','Word').withColumnRenamed('_2','Count')

countsDF.toPandas().to_csv('counts.csv', index=False)

# input_file = sc.textFile('Sample.txt')

# counts = input_file.flatMap(lambda line: line.split()).map(lambda word: (word, 1)) \
#                  .reduceByKey(lambda a, b: a + b)


# countsDF = sqlContext.createDataFrame(counts) \
#             .withColumnRenamed('_1','Word').withColumnRenamed('_2','Count')

# countsDF.toPandas().to_csv('counts.csv', index=False)

# countsDF.show()

NameError: ignored